In [40]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import requests
from bs4 import BeautifulSoup
from selenium import webdriver

from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import time
from tqdm import tqdm

import warnings
warnings.filterwarnings('ignore')

driver_pathname = 'C:/Users/sujung/Downloads/chromedriver_win32/chromedriver.exe'
# driver_pathname = 'C:/Users/yonsai/Downloads/chromedriver_win32/chromedriver.exe'


In [41]:
df = pd.read_csv('data/멜론테그번호.CSV', encoding='euc-kr')
print(df.shape)
df.head()

(45, 2)


,tag_name,id
0,가을,2
1,기분전환,3
2,휴식,4
3,힐링,5
4,사랑,6


In [42]:
tag_list = df.id.values
# 메인 tag URL
tag_url1 = 'https://www.melon.com/dj/tag/djtaghub_list.htm?tagSeq='
tag_url2 = '#params%5BtagSeq%5D='
tag_url3 = '&params%5BorderBy%5D=POP&po=pageObj&startIndex='
# PlayList URL
playlist_url = 'https://www.melon.com/mymusic/dj/mymusicdjplaylistview_inform.htm?plylstSeq='

header = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36'}
# driver = webdriver.Chrome('C:/Users/yonsai/Downloads/chromedriver_win32/chromedriver.exe')



In [44]:
plist1, plist2 = [], []

is_plist = []
song_ids = []

driver = webdriver.Chrome(driver_pathname)

for i in tqdm(tag_list):

    
    for k in range(1, 182, 20):
        try : 
            driver.get(f'{tag_url1}{i}{tag_url2}{i}{tag_url3}{k}')
            time.sleep(2)

            soup = BeautifulSoup(driver.page_source, 'html.parser')
            lis = soup.select('.service_list_play.d_djcol_list > ul > li > div > a')
            plist = [li['href'].split(',')[-1][1:-3] for li in lis]

            for pli in plist:

                if pli in is_plist:
                    continue

                is_plist.append(pli)

                # print(playlist_url + pli)
                driver.get(playlist_url + pli)
                time.sleep(1)

                soup = BeautifulSoup(driver.page_source, 'html.parser')

                # # plist1에 들어갈 요소들
                # title = soup.select_one('.ellipsis.song_name').text.strip()
                # # print(title)
                # tag = soup.select_one('.tag_list.type03').text.replace('\n', ' ')
                # like = int(soup.select_one('.cnt').text.split()[-1].replace(',', '').strip())
                
                trs = soup.select('.service_list_song.d_song_list > table > tbody > tr' )
                # song_ids = []
                # plist2에 들어갈 요소들
                for tr in trs:
                    
                    if tr.select_one('.bullet_icons.age_19') :  # 19금 제외
                        continue

                    img = tr.select_one('img')['src']

                    tmp = tr.select_one('.rank01 > span > a')
                    s_id = tmp['href'].split(',')[-1][:-2]

                    if s_id in song_ids:
                        continue

                    song_ids.append(s_id)

                    s_title = tmp.text.strip()
                    s_artist = tr.select_one('.rank02 > span').text
                    s_album = tr.select_one('.rank03 > a').text

                    plist2.append({'songId': s_id, 'title': s_title, 'artist': s_artist, 'img': img, 'album' : s_album})
                    # plist2.append({'plylstSeq': pli, 'songId': s_id, 'title': s_title, 'artist': s_artist, 'img': img})
                
                # plist1.append({'plylstSeq': pli, 'title': title, 'tag': tag, 'like':like, 'songIds': ' '.join(song_ids)})
        except:
            continue

# df1 = pd.DataFrame(plist1)
df2 = pd.DataFrame(plist2)

print(df2.shape)

driver.close()


100%|██████████| 45/45 [3:49:17<00:00, 305.72s/it]  


(90153, 5)


In [45]:
df2.to_csv('data/plist2.csv', index=False)

In [46]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90153 entries, 0 to 90152
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   songId  90153 non-null  object
 1   title   90153 non-null  object
 2   artist  90153 non-null  object
 3   img     90153 non-null  object
 4   album   90153 non-null  object
dtypes: object(5)
memory usage: 3.4+ MB


In [79]:
df_song = pd.read_csv('web/static/data/melon_song_v3.csv')
df_song.songId = df_song.songId.astype(str)

df_old_p2 = pd.read_csv('web/static/data/melon_playlist2_v2.csv')
df_old_p2.songId = df.songId.astype(str)

In [71]:
df_old_p2.shape, df_song.shape

((60271, 4), (6243, 14))

In [76]:
p2_songIds = df_old_p2.songId.values
songIds = df_song.songId.values

In [77]:
df2[df2.songId.isin(p2_songIds)]

,songId,title,artist,img,album
0,33213267,Love Song,Why Don't We,https://cdnimg.melon.co.kr/cm2/album/images/10...,The Good Times and The Bad Ones
1,32544401,Ghost In A Flower,Yorushika,https://cdnimg.melon.co.kr/cm2/album/images/10...,Ghost In A Flower
2,33502105,Every Second,Mina Okabe,https://cdnimg.melon.co.kr/cm2/album/images/10...,Every Second
3,33469725,Loving You Girl (Feat. Hkeem),Peder Elias,https://cdnimg.melon.co.kr/cm2/album/images/10...,Love & Loneliness
4,32464068,SKYLINE,김세정,https://cdnimg.melon.co.kr/cm2/album/images/10...,화분
...,...,...,...,...,...
90025,2127822,1517,The Whitest Boy Alive,https://cdnimg.melon.co.kr/cm/album/images/005...,Rules
90063,30499843,Like A Woman,Kacy Hill,https://cdnimg.melon.co.kr/cm/album/images/100...,Like A Woman
90114,508723,Battle Without Honor Or Humanity,Hotei Tomoyasu,https://cdnimg.melon.co.kr/cm/album/images/000...,Kill Bill Vol. 1 Original Soundtrack (PA Version)
90116,4200466,Beautiful People,Sam Ock,https://cdnimg.melon.co.kr/cm/album/images/021...,Simple Steps (Deluxe Edition)


In [74]:
df2[df2.songId.isin(songIds)]

,songId,title,artist,img,album
17,35745727,Love Song,"원슈타인, 조이 (JOY)",https://cdnimg.melon.co.kr/cm2/album/images/11...,World Peace Project Vol.2
26,35018129,Vancouver,BIG Naughty (서동현),https://cdnimg.melon.co.kr/cm2/album/images/10...,Lovey Dovey
28,35337708,Alone (Feat. 이하이),쿠기 (Coogie),https://cdnimg.melon.co.kr/cm2/album/images/10...,ALONE
36,4583488,작은별,AKMU (악뮤),https://cdnimg.melon.co.kr/cm/album/images/022...,PLAY
37,34927767,정이라고 하자 (Feat. 10CM),BIG Naughty (서동현),https://cdnimg.melon.co.kr/cm2/album/images/10...,정이라고 하자
...,...,...,...,...,...
86384,31286161,사이렌 (Siren),선미,https://cdnimg.melon.co.kr/cm/album/images/102...,WARNING
86967,1611502,Returns,엠씨더맥스 (M.C the MAX),https://cdnimg.melon.co.kr/cm/album/images/003...,Returns
87241,3573175,연애시대 (Feat. Ra.D) (Narr. 한효주),이승기,https://cdnimg.melon.co.kr/cm/album/images/020...,연애시대
87242,3488750,Hate You,2NE1,https://cdnimg.melon.co.kr/cm/album/images/013...,2NE1 2nd Mini Album


In [75]:
df[df.songId.isin(songIds)]

,songId,title,artist,img
23,35018129,Vancouver,BIG Naughty (서동현),https://cdnimg.melon.co.kr/cm2/album/images/10...
25,35337708,Alone (Feat. 이하이),쿠기 (Coogie),https://cdnimg.melon.co.kr/cm2/album/images/10...
33,4583488,작은별,AKMU (악뮤),https://cdnimg.melon.co.kr/cm/album/images/022...
34,34927767,정이라고 하자 (Feat. 10CM),BIG Naughty (서동현),https://cdnimg.melon.co.kr/cm2/album/images/10...
40,31674790,지켜줄게,백예린 (Yerin Baek),https://cdnimg.melon.co.kr/cm/album/images/102...
...,...,...,...,...
59087,35702661,다시 사랑한다면(풍락),풍락,https://cdnimg.melon.co.kr/cm2/album/images/11...
59262,33234463,걷고 싶다,임영웅,https://cdnimg.melon.co.kr/cm2/album/images/10...
59833,36335569,좆됐다 좆됐어,천진우,https://cdnimg.melon.co.kr/cm2/album/images/11...
59966,34183979,인생 한잔,정은지,https://cdnimg.melon.co.kr/cm2/album/images/10...


In [80]:
df_new = df2
is_songIds = df_new.songId.values

In [82]:
df_old_p2.shape

(60271, 4)

In [86]:
# 새로받은(is_songIds)가 플레이 리스트2에 들어 있는 songId
# 이것들을 제외한 songId 의 앨범 정보를 받는다.
is_old_songIds = df_old_p2[df_old_p2.songId.isin(is_songIds)].songId.values

In [89]:
df_plist1 = pd.read_csv('web/static/data/melon_playlist1.csv')
df_plist1.plylstSeq = df_plist1.plylstSeq.astype(str)
df_plist1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3845 entries, 0 to 3844
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   plylstSeq  3845 non-null   object
 1   title      3845 non-null   object
 2   tag        3845 non-null   object
 3   like       3845 non-null   int64 
 4   songIds    3845 non-null   object
dtypes: int64(1), object(4)
memory usage: 150.3+ KB


In [93]:

is_plist = []
song_ids = list(is_old_songIds)
plist2_new = []

driver = webdriver.Chrome(driver_pathname)

for pli in tqdm(df_plist1.plylstSeq.values):

    if pli in is_plist:
        continue

    is_plist.append(pli)

    # print(playlist_url + pli)
    driver.get(playlist_url + pli)
    time.sleep(1)

    try:
        soup = BeautifulSoup(driver.page_source, 'html.parser')
    except:
        continue

    # # plist1에 들어갈 요소들
    # title = soup.select_one('.ellipsis.song_name').text.strip()
    # # print(title)
    # tag = soup.select_one('.tag_list.type03').text.replace('\n', ' ')
    # like = int(soup.select_one('.cnt').text.split()[-1].replace(',', '').strip())
    
    try:
        trs = soup.select('.service_list_song.d_song_list > table > tbody > tr' )
    except:
        continue

    # song_ids = []
    # plist2에 들어갈 요소들
    for tr in trs:
        
        if tr.select_one('.bullet_icons.age_19') :  # 19금 제외
            continue

        img = tr.select_one('img')['src']

        tmp = tr.select_one('.rank01 > span > a')
        s_id = tmp['href'].split(',')[-1][:-2]

        if s_id in song_ids:
            continue

        song_ids.append(s_id)

        s_title = tmp.text.strip()
        s_artist = tr.select_one('.rank02 > span').text
        s_album = tr.select_one('.rank03 > a').text

        plist2_new.append({'songId': s_id, 'title': s_title, 'artist': s_artist, 'img': img, 'album' : s_album})


df_plist2_new = pd.DataFrame(plist2_new)


print(df_plist2_new.shape)

driver.close()

100%|██████████| 3845/3845 [2:35:32<00:00,  2.43s/it]  


(26027, 5)


In [94]:
df_plist2_new.to_csv('data/plist2_new.csv', index=False)

In [95]:
df_plist2 = pd.concat([df_new, df_plist2_new])
print(df_plist2.shape)
df_plist2.shape


(116180, 5)


(116180, 5)

In [98]:
df_plist2.songId.nunique()

115551

In [99]:
df_plist2.drop_duplicates(subset=['songId'], inplace=True)
df_plist2.shape, df_plist2.songId.nunique()

((115551, 5), 115551)

In [102]:
df_old_p2
p2_songIds = df_old_p2.songId.values

In [104]:
df_plist2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 115551 entries, 0 to 26026
Data columns (total 5 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   songId  115551 non-null  object
 1   title   115551 non-null  object
 2   artist  115551 non-null  object
 3   img     115551 non-null  object
 4   album   115551 non-null  object
dtypes: object(5)
memory usage: 5.3+ MB


In [106]:
df_old_p2.shape

(60271, 4)

In [105]:
df_plist2[df_plist2.songId.isin(p2_songIds)]

,songId,title,artist,img,album
0,33213267,Love Song,Why Don't We,https://cdnimg.melon.co.kr/cm2/album/images/10...,The Good Times and The Bad Ones
1,32544401,Ghost In A Flower,Yorushika,https://cdnimg.melon.co.kr/cm2/album/images/10...,Ghost In A Flower
2,33502105,Every Second,Mina Okabe,https://cdnimg.melon.co.kr/cm2/album/images/10...,Every Second
3,33469725,Loving You Girl (Feat. Hkeem),Peder Elias,https://cdnimg.melon.co.kr/cm2/album/images/10...,Love & Loneliness
4,32464068,SKYLINE,김세정,https://cdnimg.melon.co.kr/cm2/album/images/10...,화분
...,...,...,...,...,...
26022,33651584,Let Me In (20 CUBE) [Japanese Ver.],ENHYPEN,https://cdnimg.melon.co.kr/cm2/album/images/10...,BORDER : Hakanai
26023,33651585,Forget Me Not,ENHYPEN,https://cdnimg.melon.co.kr/cm2/album/images/10...,BORDER : Hakanai
26024,32038887,Indigo Night (Feat. Xydo),Wasp,https://cdnimg.melon.co.kr/cm2/album/images/10...,Sunshine Wave
26025,31859093,TRIP,City Nature,https://cdnimg.melon.co.kr/cm/album/images/102...,TRIP


In [108]:
df_old_p2[~df_old_p2.songId.isin(df_plist2.songId.values)]

,songId,title,artist,img
4006,34774983,Winnin',링크 (Link),https://cdnimg.melon.co.kr/cm2/album/images/10...
4476,36461318,그런 말은 쉽게 하는 게 아냐,"유리(YURI), MC 스나이퍼",https://cdnimg.melon.co.kr/cm2/album/images/11...
4550,36243335,귀여운 질투,신민용,https://cdnimg.melon.co.kr/cm2/album/images/11...
4564,30517895,한걸음 더 가까이,Various Artists,https://cdnimg.melon.co.kr/cm/album/images/100...
4567,30154697,Chrysler Bldg.,Thomas Newman,https://cdnimg.melon.co.kr/cm/album/images/100...
...,...,...,...,...
60078,36468772,그럼에도 불구하고,"알레프 (ALEPH), Mingginyu (밍기뉴)",https://cdnimg.melon.co.kr/cm2/album/images/11...
60083,36465429,순간,정훈,https://cdnimg.melon.co.kr/cm2/album/images/11...
60084,36467720,"인아, 종현 그리고 우리",인아,https://cdnimg.melon.co.kr/cm2/album/images/11...
60087,36467803,너의 별,윤예담,https://cdnimg.melon.co.kr/cm2/album/images/11...


In [109]:
plist1 = pd.read_csv('data/plist2.csv')
plist2 = pd.read_csv('data/plist2_new.csv')
plist = pd.concat([plist1, plist2])
print(plist.shape)
plist.drop_duplicates(subset=['songId'], inplace=True)
print(plist.shape)


(116180, 5)
(115551, 5)


In [110]:
plist.to_csv('data/plist2_total.csv', index=False)

### 여기서 부터

In [149]:
plist2 = pd.read_csv('web/static/data/melon_playlist2_v2.csv')
plist2_new = pd.read_csv('data/plist2_total.csv')

In [140]:
plist2.shape, plist2_new.shape

((60271, 4), (115551, 5))

In [141]:
plist2_new.head(1)

,songId,title,artist,img,album
0,33213267,Love Song,Why Don't We,https://cdnimg.melon.co.kr/cm2/album/images/10...,The Good Times and The Bad Ones


In [142]:
plist2.head(1)

,songId,title,artist,img
0,33213267,Love Song,Why Don't We,https://cdnimg.melon.co.kr/cm2/album/images/10...


In [143]:
df2 = pd.merge(plist2, plist2_new[['songId', 'album']], how='left', on='songId')

In [144]:
df2.songId.nunique()

60271

In [145]:
df2.head()

,songId,title,artist,img,album
0,33213267,Love Song,Why Don't We,https://cdnimg.melon.co.kr/cm2/album/images/10...,The Good Times and The Bad Ones
1,32544401,Ghost In A Flower,Yorushika,https://cdnimg.melon.co.kr/cm2/album/images/10...,Ghost In A Flower
2,33502105,Every Second,Mina Okabe,https://cdnimg.melon.co.kr/cm2/album/images/10...,Every Second
3,33469725,Loving You Girl (Feat. Hkeem),Peder Elias,https://cdnimg.melon.co.kr/cm2/album/images/10...,Love & Loneliness
4,32464068,SKYLINE,김세정,https://cdnimg.melon.co.kr/cm2/album/images/10...,화분


In [132]:
songIds = df[df.album.isna()].songId.values

In [133]:
len(songIds)

551

### 앨범 없는 곡들 가져오기

In [137]:
driver = webdriver.Chrome(driver_pathname)

infos = []
for songId in tqdm(songIds):
    song_url = f'https://www.melon.com/song/detail.htm?songId={songId}'
    try:
        # print(song_url)

        driver.get(song_url)
        time.sleep(2)

        soup = BeautifulSoup(driver.page_source, 'html.parser')

        info = soup.select_one('#conts')

        title = info.select_one('.song_name').get_text().strip().split('\t')[-1]
        artist = info.select_one('.artist_name').get_text()
        album = info.select_one('.list > dd').get_text()
        img = info.select_one('.image_typeAll > img')['src']
        # print(songId, title, artist, img, album)
        infos.append({'songId': songId, 'title': title, 'artist': artist, 'img': img, 'album': album })

    except:
        continue

driver.close()

df = pd.DataFrame(infos)
df.shape

100%|██████████| 551/551 [25:32<00:00,  2.78s/it]


(544, 5)

In [138]:
df.to_csv('data/plist2_other.csv', index=False)

In [146]:
df

,songId,title,artist,img,album
0,34774983,Winnin',링크 (Link),https://cdnimg.melon.co.kr/cm2/album/images/10...,NONSTOP
1,36461318,그런 말은 쉽게 하는 게 아냐,유리(YURI),https://cdnimg.melon.co.kr/cm2/album/images/11...,그런 말은 쉽게 하는 게 아냐
2,36243335,귀여운 질투,신민용,https://cdnimg.melon.co.kr/cm2/album/images/11...,일타 스캔들 OST
3,30517895,한걸음 더 가까이,남혜승,https://cdnimg.melon.co.kr/cm/album/images/100...,수상한 파트너 OST
4,30154697,Chrysler Bldg.,Thomas Newman,https://cdnimg.melon.co.kr/cm/album/images/100...,패신저스 OST
...,...,...,...,...,...
539,36468772,그럼에도 불구하고,알레프 (ALEPH),https://cdnimg.melon.co.kr/cm2/album/images/11...,사과향
540,36465429,순간,정훈,https://cdnimg.melon.co.kr/cm2/album/images/11...,순간
541,36467720,"인아, 종현 그리고 우리",인아,https://cdnimg.melon.co.kr/cm2/album/images/11...,"인아, 종현 그리고 우리"
542,36467803,너의 별,윤예담,https://cdnimg.melon.co.kr/cm2/album/images/11...,너의 별


In [150]:

plist2_new.head()

,songId,title,artist,img,album
0,33213267,Love Song,Why Don't We,https://cdnimg.melon.co.kr/cm2/album/images/10...,The Good Times and The Bad Ones
1,32544401,Ghost In A Flower,Yorushika,https://cdnimg.melon.co.kr/cm2/album/images/10...,Ghost In A Flower
2,33502105,Every Second,Mina Okabe,https://cdnimg.melon.co.kr/cm2/album/images/10...,Every Second
3,33469725,Loving You Girl (Feat. Hkeem),Peder Elias,https://cdnimg.melon.co.kr/cm2/album/images/10...,Love & Loneliness
4,32464068,SKYLINE,김세정,https://cdnimg.melon.co.kr/cm2/album/images/10...,화분


In [152]:
df.head()

,songId,title,artist,img,album
0,34774983,Winnin',링크 (Link),https://cdnimg.melon.co.kr/cm2/album/images/10...,NONSTOP
1,36461318,그런 말은 쉽게 하는 게 아냐,유리(YURI),https://cdnimg.melon.co.kr/cm2/album/images/11...,그런 말은 쉽게 하는 게 아냐
2,36243335,귀여운 질투,신민용,https://cdnimg.melon.co.kr/cm2/album/images/11...,일타 스캔들 OST
3,30517895,한걸음 더 가까이,남혜승,https://cdnimg.melon.co.kr/cm/album/images/100...,수상한 파트너 OST
4,30154697,Chrysler Bldg.,Thomas Newman,https://cdnimg.melon.co.kr/cm/album/images/100...,패신저스 OST


In [153]:
df3 = pd.concat([plist2_new, df])
df3.shape

(116095, 5)

In [157]:
df3.drop_duplicates(subset=['songId'], inplace=True)

In [158]:
df4 = pd.merge(plist2, df3[['songId', 'album']], how='left', on='songId')
df4.shape, plist2.shape

((60271, 5), (60271, 4))

In [159]:
df4.head()

,songId,title,artist,img,album
0,33213267,Love Song,Why Don't We,https://cdnimg.melon.co.kr/cm2/album/images/10...,The Good Times and The Bad Ones
1,32544401,Ghost In A Flower,Yorushika,https://cdnimg.melon.co.kr/cm2/album/images/10...,Ghost In A Flower
2,33502105,Every Second,Mina Okabe,https://cdnimg.melon.co.kr/cm2/album/images/10...,Every Second
3,33469725,Loving You Girl (Feat. Hkeem),Peder Elias,https://cdnimg.melon.co.kr/cm2/album/images/10...,Love & Loneliness
4,32464068,SKYLINE,김세정,https://cdnimg.melon.co.kr/cm2/album/images/10...,화분


In [160]:
df4.isna().sum()

songId    0
title     0
artist    0
img       0
album     9
dtype: int64

In [161]:
df4[df4.album.isna()]

,songId,title,artist,img,album
5823,32363372,화면,Oowl Hannal (우울 한날),https://cdnimg.melon.co.kr/cm2/album/images/10...,NaN
8558,35667403,Here With Me,d4vd,https://cdnimg.melon.co.kr/cm2/album/images/11...,NaN
8871,3013124,빗속에서,존박,https://cdnimg.melon.co.kr/cm/album/images/010...,NaN
12650,12094789,Green Juice,Papooz,https://cdnimg.melon.co.kr/cm/album/images/029...,NaN
33529,31320979,설레도 쿨하게,홍지수 (ZISU),https://cdnimg.melon.co.kr/cm/album/images/101...,NaN
33530,31320981,첫사랑,홍지수 (ZISU),https://cdnimg.melon.co.kr/cm/album/images/101...,NaN
52021,12094785,Ann Wants to Dance,Papooz,https://cdnimg.melon.co.kr/cm/album/images/029...,NaN
52051,12094787,Toria's Song,Papooz,https://cdnimg.melon.co.kr/cm/album/images/029...,NaN
52058,12094793,Brother,Papooz,https://cdnimg.melon.co.kr/cm/album/images/029...,NaN


In [169]:
df4.iloc[5823,4] = 'null'

In [171]:
df4.iloc[8871,4] = '슈퍼스타K2 Up To 11'

In [172]:
d_index = [8558, 12650, 33529, 33530, 52021, 52051, 52058]

In [175]:
list(df4[df4.album.isna()].index)

[8558, 12650, 33529, 33530, 52021, 52051, 52058]

In [176]:
df4.drop(df4.index[list(df4[df4.album.isna()].index)], inplace=True)
df4.shape

(60264, 5)

In [177]:
df4.isna().sum()

songId    0
title     0
artist    0
img       0
album     0
dtype: int64

In [178]:
df4.head()

,songId,title,artist,img,album
0,33213267,Love Song,Why Don't We,https://cdnimg.melon.co.kr/cm2/album/images/10...,The Good Times and The Bad Ones
1,32544401,Ghost In A Flower,Yorushika,https://cdnimg.melon.co.kr/cm2/album/images/10...,Ghost In A Flower
2,33502105,Every Second,Mina Okabe,https://cdnimg.melon.co.kr/cm2/album/images/10...,Every Second
3,33469725,Loving You Girl (Feat. Hkeem),Peder Elias,https://cdnimg.melon.co.kr/cm2/album/images/10...,Love & Loneliness
4,32464068,SKYLINE,김세정,https://cdnimg.melon.co.kr/cm2/album/images/10...,화분


In [ ]:
df4.to_csv('data/melon_playlist2_v3.csv', index=False)